In [ ]:
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2

# 构造参数解析器并解析参数
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--images", type=str, required=True,
                help="path to input directory of images to stitch")
ap.add_argument("-o", "--output", type=str, required=True,
                help="path to the output image")
ap.add_argument("-c", "--crop", type=int, default=0,
                help="whether to crop out largest rectangular region")
args = vars(ap.parse_args())

# 获取输入图像的路径并初始化图像列表
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(args["images"])))

images = []

# 遍历图像路径，加载每个图像，并将它们添加到图像列表中
for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	images.append(image)

# 初始化OpenCV的图像拼接对象，然后执行图像拼接
print("[INFO] stitching images...")
stitcher = cv2.createStitcher() if imutils.is_cv3() else cv2.Stitcher_create()
(status, stitched) = stitcher.stitch(images)

# 如果状态为0，则表示OpenCV成功执行了图像拼接
if status == 0:
    # 检查是否需要裁剪拼接后的图像的最大矩形区域

    if args["crop"] > 0:
        # 在拼接后的图像周围创建一个10像素的边框
        print("[INFO] cropping...")
        stitched = cv2.copyMakeBorder(stitched, 2, 2, 2, 2,
                                      cv2.BORDER_CONSTANT, (0, 0, 0))

        # 将拼接后的图像转换为灰度图像，并进行阈值处理，
        # 将大于零的像素设置为255（前景），其他像素保持为0（背景）
        gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

        # 在阈值图像中找到所有外部轮廓，然后找到最大的轮廓，
        # 该轮廓将是拼接后图像的轮廓/外形
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)

        # 分配内存给掩膜，该掩膜将包含拼接后图像区域的矩形边界框
        mask = np.zeros(thresh.shape, dtype="uint8")
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

        # 创建掩膜的两个副本：一个用作实际的最小矩形区域，
        # 另一个用作计数器，用于统计需要删除的像素数以形成最小矩形区域
        minRect = mask.copy()
        sub = mask.copy()

        # 循环直到减法图像中没有非零像素为止
        while cv2.countNonZero(sub) > 0:
            # erode the minimum rectangular mask and then subtract
            # the thresholded image from the minimum rectangular mask
            # so we can count if there are any non-zero pixels left
            minRect = cv2.erode(minRect, None)
            sub = cv2.subtract(minRect, thresh)

        # 在最小矩形掩膜中找到轮廓，然后提取边界框（x，y）坐标
        cnts = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        (x, y, w, h) = cv2.boundingRect(c)

        # 从拼接后的图像中提取裁剪区域，并将其保存为拼接后的图像
        stitched = stitched[y:y + h, x:x + w]

    # 将拼接后的图像保存到指定的输出路径
    cv2.imwrite(args["output"], stitched)

    # 使用OpenCV显示拼接后的图像
    cv2.imshow("Stitched", stitched)
    cv2.waitKey(0)

# 如果状态不为0，则表示OpenCV未能成功执行图像拼接
else:
    print("[INFO] image stitching failed ({})".format(status))